In [1]:
#######import necessary packages
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.plotly as py #this package is for visualization
import plotly.graph_objs as go #this package is for visualization
import pandas as pd #this package is for handling the dataframe
from pandasql import sqldf  #This is to allow users to use SQL queries to create a new dataframe
pysqldf = lambda q: sqldf(q, globals())

In [2]:
df = pd.read_excel('/export.xls') #read the file in


In [3]:
"""Rename the columns so that they are easier to handle without blank space in between"""
df = df.rename(columns={'Nike, Inc. Brand(s)': 'Brands', '% Female Workers':'FemaleWorkers', 
                        'Total Workers':'TotalWorkers', 'Factory Type': 'FactoryType', 'Product Type':'ProductType', 
                        'Supplier Group':'SupplierGroup','Factory Name':'FactoryName'})
df.head()

,FactoryName,FactoryType,ProductType,Brands,Events,SupplierGroup,Address,City,State,Postal Code,...,Operating Region,TotalWorkers,Line Workers,FemaleWorkers,% Migrant Workers,Contact Name,Contact Phone,Contact Fax,Contact Email,Subcons
0,"A & K DESIGNS, INC.",FINISHED GOODS,APPAREL,NIKE,Collegiate,A & K DESIGNS,8325 SE HARNEY ST.,PORTLAND,OREGON,97266,...,NaN,108.0,94.0,70.0%,0.0%,Jenny Foster,971-634-1901,877-968-2889,jenny.foster@akdesignsinc.com,NaN
1,A. FIRST VINA CO. LTD.,FINISHED GOODS,APPAREL,"NIKE, CONVERSE, HURLEY",NaN,A. FIRST VINA CO. LTD.,TAM PHUOC INDUSTRIAL ZONE,BIEN HOA,DONG NAI,n/a,...,NaN,1958.0,1844.0,88.0%,6.2%,NaN,NaN,NaN,NaN,NaN
2,"ACABADOS Y SERIGRAFIA, SA (ACSA)",FINISHED GOODS,APPAREL,HURLEY,NaN,NO SUPPLIER GROUP,KM.16.5 A SAN JUAN SACATEPEQUEZ,MIXCO,GUATEMALA,NaN,...,NaN,140.0,130.0,31.0%,0.0%,NaN,NaN,NaN,NaN,NaN
3,ADORA FOOTWEAR LIMITED,FINISHED GOODS,FOOTWEAR,CONVERSE,NaN,HONG FU,TAM DIEP INDUSTRIAL ZONE,TAM DIEP TOWN,NINH BINH PROVINCE.,NaN,...,NaN,7678.0,6736.0,90.0%,14.0%,NaN,NaN,NaN,NaN,NaN
4,AIRTEX,FINISHED GOODS,EQUIPMENT,NIKE,NaN,"STX, LLC","1620 BROADWAY, NE",MINNEAPOLIS,MINNESOTA,55413,...,NaN,72.0,42.0,69.0%,0.0%,NaN,NaN,NaN,NaN,NaN


In [4]:
"""This section is to process data and prepare tables for the next step"""
def generate_table(dataframe, max_rows=10): #a function to design tables
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )
"""A function to create a table that includes the count of factories and suppliers by factory types"""
def by_factorytype_count():
    factorytype = """SELECT FactoryType, Count(DISTINCT FactoryName) as FactoryCount, Count(DISTINCT SupplierGroup) as SupplierCount From df Group By FactoryType"""
    factory_df = pysqldf(factorytype)
    return factory_df

"""A function to create a table that includes the count of factories and suppliers by product types"""
def by_producttype_count():
    producttype = """SELECT ProductType, Count(DISTINCT FactoryName) as FactoryCount, Count(DISTINCT SupplierGroup) as SupplierCount From df Group By ProductType"""
    product_df = pysqldf(producttype)
    return product_df

"""A function to create a table that includes a summary of total workers, % of female workers in the workforce
            the number of workers in one factory on average, the count of suppliers and factories by brands"""
def summary_by_brands():
    summary = """SELECT Brands, Sum(TotalWorkers) as TotalWorkers, Round(AVG(TotalWorkers),0) as AvgTotalWorkers, Round(AVG(FemaleWorkers),0) as Percentage_of_FemaleWorkers, Count(DISTINCT SupplierGroup) as SupplierTotal, Count(DISTINCT FactoryName) as FactoryTotal From df Group By Brands""" #create a data frame using a SQL
    summary_df = pysqldf(summary)
    return summary_df

In [ ]:
app = dash.Dash('') #create a Dash app
#add layout
app.layout = html.Div([
    html.Div([
        html.H4('The number of factories and suppliers by factory types') #add the first table
    ]),
    html.Div(generate_table(by_factorytype_count())),
    html.Br(),
    html.Div([
        html.H4('The number of factories and suppliers by product types') #add the second table
    ]),    
    html.Div(generate_table(by_producttype_count())),
    html.Br(),
    html.Div([
        html.H4('Summary by Brands') #add the second table
    ]),
    html.Br(),
    html.Div(generate_table(summary_by_brands()))
])

app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"}) #add css to make tables look prettier

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Oct/2018 10:17:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2018 10:17:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2018 10:17:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2018 10:21:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2018 10:21:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2018 10:21:20] "GET / HTTP/1.1" 200 -
